In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn import datasets

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import models
from tensorflow.keras import layers

import pandas as pd

from imblearn.under_sampling import RandomUnderSampler

2022-11-28 19:59:55.511408: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 19:59:55.627265: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-28 19:59:56.151857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-28 19:59:56.151914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

Na początku wczytano zbiór danych:

In [2]:
cars = pd.read_csv('car.data', encoding='utf-8')
cars

,buying,maint,doors,persons,lug_boot,safety,accept
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


Następnie zamieniono dane w postaci ciągu znaków na dane liczbowe, w celu przystosowania danych do przyjęcia przez sieć neuronową:

In [3]:
cars['buying'].replace(['low', 'med', 'high', 'vhigh'], [0,1,2,3], inplace=True)
cars['maint'].replace(['low', 'med', 'high', 'vhigh'], [0,1,2,3], inplace=True)
cars['doors'].replace(['2', '3', '4', '5more'], [0,1,2,3], inplace=True)
cars['persons'].replace(['2', '4', 'more'], [0,1,2], inplace=True)
cars['lug_boot'].replace(['small', 'med', 'big'], [0,1,2], inplace=True)
cars['safety'].replace(['low', 'med', 'high'], [0,1,2], inplace=True)
cars['accept'].replace(['unacc', 'acc', 'good', 'vgood'], [0,1,2,3], inplace=True)

Następnie wypisano liczność każdej z klas decyzyjnych:

In [4]:
cars["accept"].value_counts()

0    1210
1     384
2      69
3      65
Name: accept, dtype: int64

Kolejnym krokiem było wyodrębnienie parametrów i klas decyzyjnych, a następnie podział zbioru na dane treningowe i testowe:

In [5]:
from sklearn.model_selection import train_test_split

x = cars.drop("accept", axis=1)
x

,buying,maint,doors,persons,lug_boot,safety
0,3,3,0,0,0,0
1,3,3,0,0,0,1
2,3,3,0,0,0,2
3,3,3,0,0,1,0
4,3,3,0,0,1,1
...,...,...,...,...,...,...
1723,0,0,3,2,1,1
1724,0,0,3,2,1,2
1725,0,0,3,2,2,0
1726,0,0,3,2,2,1


In [6]:
y = cars["accept"]
y

0       0
1       0
2       0
3       0
4       0
       ..
1723    2
1724    3
1725    0
1726    2
1727    3
Name: accept, Length: 1728, dtype: int64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

Po udanym podziale danych zastosowano undersampling zbioru:

In [8]:
sampling_strategy = {0: 570, 1: 281, 2: 54, 3: 45}
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy)

X_smote, y_smote = undersampler.fit_resample(X_train, y_train)

Nowo powstały zbiór miał tylko 949 rekordów, w odróżnieniu oryginalnego o liczności 1727:

In [9]:
X_smote

,buying,maint,doors,persons,lug_boot,safety
0,1,0,1,0,2,0
1,3,3,1,0,0,0
2,2,0,1,2,0,0
3,2,3,3,0,2,1
4,0,0,0,2,0,1
...,...,...,...,...,...,...
945,0,1,2,1,2,2
946,1,1,3,2,2,2
947,0,2,3,1,1,2
948,1,0,2,1,2,2


In [10]:
y_smote

0      0
1      0
2      0
3      0
4      0
      ..
945    3
946    3
947    3
948    3
949    3
Name: accept, Length: 950, dtype: int64

In [11]:
y_smote.value_counts()

0    570
1    281
2     54
3     45
Name: accept, dtype: int64

Następnie zastosowano walidację krzyżową z zastosowaniem klasyfikatora 5-fold:

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score

clf = DecisionTreeClassifier(random_state=42)

k_folds = KFold(n_splits = 5)

scores = cross_val_score(clf, X_smote, y_smote, cv=k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.98421053 0.98421053 0.97894737 0.87894737 0.47368421]
Average CV Score:  0.86
Number of CV Scores used in Average:  5


Średni wynik walidacji krzyżowej wyniósł 0.86, jest to wynik zadowalający.

# Wnioski

Podczas realizowania zadania przygotowano zbiór danych dotyczących akceptacji samochodu do dalszego etapu analizy danych, czyli uczenia modelu. Ustalono, że analizowany zbiór danych jest kompletny tzn. że nie trzeba było usuwać błędnych rekordów, duplikatów oraz brakujących wartości. Ponadto zamieniono wartości typu string na wartości liczbowe w celu dalszej obróbki danych oraz zaimplementowano walidację krzyżową, która dała pozytywny rezultat. Nie napotkano większych problemów z realizacją zadania.